In [1]:
!pip install pyspark==3.4.2

In [2]:
!pip install minio

In [3]:
!pip install delta-spark==2.4.0

In [4]:
from pyspark.sql import SparkSession
from delta import *
from minio import Minio
import pyspark.sql.functions as F
import os
import shutil

In [5]:
spark = (
    SparkSession.builder.master("local[*]").appName("csvToParquet")
    #HADOOP
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000")
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin")
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
    #DELTA
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.jars", "jars/aws-java-sdk-bundle-1.11.1026.jar, \
            jars/hadoop-aws-3.2.0.jar")
    .getOrCreate()
)


24/09/05 19:14:16 WARN Utils: Your hostname, DESKTOP-CV9TLA8 resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/09/05 19:14:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/hacs/.cache/pypoetry/virtualenvs/pyspark-delta-gPM1PAZH-py3.10/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/hacs/.ivy2/cache
The jars for the packages stored in: /home/hacs/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b6824135-4496-45e3-8a71-5a24718ef24e;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 165ms :: artifacts dl 4ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0

In [6]:
client=Minio("localhost:9000",
             access_key="minioadmin",
             secret_key="minioadmin",
             secure=False
            )


In [7]:
objects=client.list_objects('raw', prefix='CAPES/')

In [8]:
objects=client.list_objects('raw', prefix='CAPES/')
csvs = []
for object in objects:
    csvs.append(object.object_name)

In [10]:
#FUNÇÃO PARA LISTAR AS COLUNAS IGUAIS DOS DOIS DATAFRAMES
def equalslist(df1, df2):
    return list(set(df1.columns).intersection(set(df2.columns)))

#FUNÇÃO PARA JUNTAR DOIS DATAFRAMES, ESPECIFICANDO A KEY USADA E O MÉTODO DE JUNÇÃO 
def dfjoin(dataframe1, dataframe2, key, typejoin):
    
    equals = equalslist(dataframe1,dataframe2)
    equals.remove(key)

    #USANDO UM DATAFRAME AUXILIAR PARA NÃO MODIFICAR O DATAFRAME RAIZ CARREGADO
    dataframe3 = dataframe2
    for column in equals:
        dataframe3 = dataframe3.drop(column)
        
    dataframe_join = dataframe1.join(dataframe3, key, typejoin)

    #RETIRANDO AS LINHAS EM QUE A KEY É NULA
    dataframe_join = dataframe_join.dropna(subset=[key])
    return dataframe_join


In [11]:
#LISTANDO OS DATAFRAMES PRESENTES NO BUCKET RAW DO MinIO
count = 0
for csv in csvs:
    print(f'{count} - {csv}')
    count+=1

0 - CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil.csv
1 - CAPES/2021-a-2024-catalogo-de-teses-e-dissertacoes-brasil.csv
2 - CAPES/2021-a-2024-cursos-da-pos-graduacao-stricto-sensu-no-brasil.csv
3 - CAPES/2021-a-2024-detalhes-da-producao-intelectual-bibliografica-de-programas-de-pos-graduacao.csv
4 - CAPES/2021-a-2024-detalhes-da-producao-intelectual-tecnica-de-programas-de-pos-graduacao-stricto-sensu.csv
5 - CAPES/2021-a-2024-discentes-da-pos-graduacao-stricto-sensu-do-brasil.csv
6 - CAPES/2021-a-2024-docentes-da-pos-graduacao-stricto-sensu-no-brasil.csv
7 - CAPES/2021-a-2024-financiadores-de-projetos-dos-programas-de-pos-graduacao-stricto-sensu-no-brasil.csv
8 - CAPES/2021-a-2024-membros-de-projetos-dos-programas-de-pos-graduacao-stricto-sensu-no-brasil.csv
9 - CAPES/2021-a-2024-producao-intelectual-de-pos-graduacao-stricto-sensu-no-brasil.csv
10 - CAPES/2021-a-2024-programas-da-pos-graduacao-stricto-sensu-no-brasil.csv
11 - CAPE

In [26]:
#CARREGANDO TODOS OS DATAFRAMES
bucket_name = 'raw'
autores = spark.read.format('csv').option('header', 'true').option('sep', ';').load(f's3a://{bucket_name}/{csvs[0]}')
teses = spark.read.format('csv').option('header', 'true').option('sep', ';').load(f's3a://{bucket_name}/{csvs[1]}')
cursos = spark.read.format('csv').option('header', 'true').option('sep', ';').load(f's3a://{bucket_name}/{csvs[2]}')
bibliografica = spark.read.format('csv').option('header', 'true').option('sep', ';').load(f's3a://{bucket_name}/{csvs[3]}')
tecnica = spark.read.format('csv').option('header', 'true').option('sep', ';').load(f's3a://{bucket_name}/{csvs[4]}')
discentes = spark.read.format('csv').option('header', 'true').option('sep', ';').load(f's3a://{bucket_name}/{csvs[5]}')
docentes = spark.read.format('csv').option('header', 'true').option('sep', ';').load(f's3a://{bucket_name}/{csvs[6]}')
financiadores = spark.read.format('csv').option('header', 'true').option('sep', ';').load(f's3a://{bucket_name}/{csvs[7]}')
membros = spark.read.format('csv').option('header', 'true').option('sep', ';').load(f's3a://{bucket_name}/{csvs[8]}')
producoes = spark.read.format('csv').option('header', 'true').option('sep', ';').load(f's3a://{bucket_name}/{csvs[9]}')
programas = spark.read.format('csv').option('header', 'true').option('sep', ';').load(f's3a://{bucket_name}/{csvs[10]}')
projetos = spark.read.format('csv').option('header', 'true').option('sep', ';').load(f's3a://{bucket_name}/{csvs[11]}')

In [13]:
#CRIANDO O BUCKET SE ELE NÃO EXISTIR
bucket_name="bronze"
found = client.bucket_exists(bucket_name)
if not found:
    client.make_bucket(bucket_name)

In [14]:
#FUNÇÃO DE CONVERSAO PARA DELTA
def df_to_parquet(df, name_df):
    (df.write 
        .format("delta")
        .mode("overwrite")
        .save(f"delta/{name_df}")
    )
#FUNÇÃO PARA ENVIAR O DATAFRAME PARA O MinIO, CONVERTENDO EM DELTA NO PROCESSO E SALVANDO UMA VERSÃO LOCAL PARA PRESERVAR O LOG
def df_to_minio(df, bucket_name, name_df):
    #USANDO A FUNÇÃO DE CONVERSÃO EM DELTA, EM QUE A MESMA SALVA NA PASTA LOCAL DELTA COM O NOME ESPECIFICADO
    df_to_parquet(df, name_df)

    #ANDANDO PELA PASTA ESPECIFICA DO ARQUIVO DELTA E ENVIANDO-O PARA O BUCKET ESPECIFICADO
    for root, _, files in os.walk(f"delta/{name_df}"):
        for file in files:
            file_path = os.path.join(root, file)
            object_name = os.path.relpath(file_path, f"delta/{name_df}")
            object_name = os.path.join(name_df, object_name)
            client.fput_object(bucket_name, object_name, file_path)

In [15]:
#CRIANDO O PRIMEIRO DATAFRAME: PROJETOS + MEMBROS
projetos_membros_ufma = dfjoin(projetos, membros, "ID_PROJETO", "inner")

In [16]:
#FILTRO:UFMA PARA FILTRAR OS DISCENTES E DOCENTES DOS PROJETOS
projetos_membros_ufma = projetos_membros_ufma.filter(F.col("SG_ENTIDADE_ENSINO") == ("UFMA"))

In [17]:
#FILTRO:%COMPUTA% PARA FILTRAR A AREA DE ATUAÇÃO PARA CIENCIA DA COMPUTAÇÃO
projetos_membros_ufma = projetos_membros_ufma.filter(F.col("NM_AREA_AVALIACAO").like("%COMPUTA%"))

In [92]:
#CRIANDO O DATAFRAME DE PROGRAMA DOS PROJETOS DOS MEMBROS DA UFMA
programas_projetos_membros = dfjoin(projetos_membros_ufma, programas, "CD_PROGRAMA_IES", "inner") 

In [93]:
programas_projetos_membros.count()

335

In [94]:
#DIVIDINDO O DATAFRAME EM DISCENTES/DOCENTES
#CRIANDO O DATAFRAME DE PROGRAMAS DOS PROJETOS DOS DISCENTES
programas_projetos_discentes = dfjoin(programas_projetos_membros, discentes, "ID_PESSOA", "inner")
programas_projetos_discentes.count()

45

In [95]:
#E RETIRANDO AS DUPLICATAS
programas_projetos_discentes = programas_projetos_discentes.dropDuplicates()

In [96]:
#CRIANDO O DATAFRAME DE PROGRAMAS DOS PROJETOS DOS DOCENTES
programas_projetos_docentes = dfjoin(programas_projetos_membros, docentes, "ID_PESSOA", "inner")
programas_projetos_docentes.count()

586

In [97]:
#E RETIRANDO AS DUPLICATAS
programas_projetos_docentes = programas_projetos_docentes.dropDuplicates()

In [98]:
#CRIANDO OS DATAFRAMES DOS DISCENTES/DOCENTES PARA TECNICA/BIBLIOGRAFICA

#CRIANDO O DATAFRAME DE TECNICA DOS PROGRAMAS DOS PROJETOS DOS DISCENTES
detalhes_tecnica_discentes = dfjoin(programas_projetos_discentes, tecnica, "CD_PROGRAMA_IES", "inner")

#CRIANDO O DATAFRAME DE TECNICA DOS PROGRAMAS DOS PROJETOS DOS DOCENTES
detalhes_tecnica_docentes = dfjoin(programas_projetos_docentes, tecnica, "CD_PROGRAMA_IES", "inner")

#CRIANDO O DATAFRAME DE BIBLIOGRAFICA DOS PROGRAMAS DOS PROJETOS DOS DISCENTES
detalhes_bibliografica_discentes = dfjoin(programas_projetos_discentes, bibliografica, "CD_PROGRAMA_IES", "inner")

#CRIANDO O DATAFRAME DE BIBLIOGRAFICA DOS PROGRAMAS DOS PROJETOS DOS DOCENTES
detalhes_bibliografica_docentes = dfjoin(programas_projetos_docentes, bibliografica, "CD_PROGRAMA_IES", "inner")